In [ ]:
!pip install transformers torch datasets
!pip install accelerate

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/sales10.csv")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [3]:
df['quarter'] = pd.PeriodIndex(df.Date, freq='Q')
df['quarter'].head()

0    2019Q1
1    2019Q1
2    2019Q1
3    2019Q1
4    2019Q1
Name: quarter, dtype: period[Q-DEC]

In [4]:
pivot = df.pivot_table(index='quarter', values= ['Total','gross income','Quantity','cogs'], 
                       fill_value=0, aggfunc=["sum"])
pivot

sum                                
        Quantity     Total     cogs gross income
quarter                                         
2019Q1        62  4175.724  3976.88      198.844

In [8]:
# Load Pre-trained Mistral Model
from transformers import AutoTokenizer, AutoModel

# Load the Mistral model and tokenizer
#model_name = 'mistralai/mistral-7b'  # Change this to the exact model you want to use
#model_name = 'mistral:latest'
#model_name = 'llama3.2:latest'
#model_name = 'llama3.2:3b-instruct-q4_K_M'
#model_name = 'llama3.2:1b-instruct-q4_K_M'
#model_name = 'llama3.2:1b-instruct-q2_K'
model_name = 'timpal0l/mdeberta-v3-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

C:\Development\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ramezania\.cache\huggingface\hub\models--timpal0l--mdeberta-v3-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [9]:
# Prepare Data for Embedding Extraction
from datasets import load_dataset

# For example, let's use a small dataset (replace with your own dataset)
dataset = load_dataset('ag_news', split='train')

# Take a sample for embedding extraction
texts = dataset['text'][:10]

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

C:\Development\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ramezania\.cache\huggingface\hub\datasets--ag_news. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [15]:
# Tokenize the texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[    1,  9108,  1820,  ...,     0,     0,     0],
        [    1, 14645, 11812,  ...,     0,     0,     0],
        [    1, 16514,   306,  ...,     0,     0,     0],
        ...,
        [    1, 64742,  7936,  ...,     0,     0,     0],
        [    1, 40237,  6141,  ...,  3176,   261,     2],
        [    1,  9108,  1820,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [17]:
# Extract Embeddings
import torch

# Forward pass to get embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract the embeddings from the last hidden state
embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
embeddings

tensor([[[-0.2013,  0.1107,  0.4138,  ..., -0.4635, -0.7623,  0.0081],
         [-0.5379, -0.2712, -0.0335,  ...,  0.5443,  0.2244, -0.0567],
         [-0.4103, -0.1754, -0.5240,  ...,  0.0773, -0.1063, -0.3535],
         ...,
         [-0.2831,  0.1421, -0.4294,  ...,  0.2697,  0.0033, -0.8353],
         [-0.2831,  0.1421, -0.4294,  ...,  0.2697,  0.0033, -0.8353],
         [-0.2831,  0.1421, -0.4294,  ...,  0.2697,  0.0033, -0.8353]],

        [[-0.3414,  0.0687,  0.3706,  ..., -0.1908, -0.8643, -0.1546],
         [-0.3538, -0.3084, -0.2262,  ...,  0.5960, -0.0935, -0.3217],
         [-0.4751, -0.1259, -0.4441,  ..., -0.0787, -0.0325, -0.3245],
         ...,
         [-0.2831,  0.1421, -0.4294,  ...,  0.2697,  0.0033, -0.8353],
         [-0.2831,  0.1421, -0.4294,  ...,  0.2697,  0.0033, -0.8353],
         [-0.2831,  0.1421, -0.4294,  ...,  0.2697,  0.0033, -0.8353]],

        [[ 0.0276,  0.2944,  0.5794,  ..., -0.4345, -0.5545,  0.1603],
         [-0.1911,  0.2732, -0.5336,  ..., -0

In [18]:
# Get embeddings for the [CLS] token (first token)
cls_embeddings = embeddings[:, 0, :]  # Shape: (batch_size, hidden_size)
cls_embeddings

tensor([[-0.2013,  0.1107,  0.4138,  ..., -0.4635, -0.7623,  0.0081],
        [-0.3414,  0.0687,  0.3706,  ..., -0.1908, -0.8643, -0.1546],
        [ 0.0276,  0.2944,  0.5794,  ..., -0.4345, -0.5545,  0.1603],
        ...,
        [-0.3564,  0.0633,  0.5007,  ..., -0.1282, -0.7708, -0.1969],
        [-0.1750,  0.1367,  0.5979,  ..., -0.3908, -0.7538, -0.0947],
        [-0.2479,  0.1771,  0.4558,  ..., -0.4356, -0.7563,  0.1132]])

In [19]:
# Average the token embeddings (excluding padding tokens)
attention_mask = inputs['attention_mask']
mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size())
sum_embeddings = (embeddings * mask_expanded).sum(1)
avg_embeddings = sum_embeddings / mask_expanded.sum(1)
avg_embeddings

tensor([[-0.2871,  0.0090, -0.2625,  ..., -0.1699,  0.1022, -0.0614],
        [-0.3752, -0.1156, -0.4283,  ..., -0.0170,  0.1278, -0.3567],
        [-0.2869,  0.0960, -0.2552,  ..., -0.1188,  0.2940, -0.2012],
        ...,
        [-0.5288,  0.0150, -0.3822,  ..., -0.1003,  0.0169, -0.3715],
        [-0.6600,  0.1489, -0.3109,  ...,  0.0531,  0.0615, -0.4343],
        [-0.3415, -0.0648, -0.2927,  ..., -0.2367,  0.0388, -0.1358]])

In [20]:
# Fine-Tuning (Optional)
from transformers import Trainer, TrainingArguments

# Define your custom head (if required)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)  # Change the number of labels

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,  # Provide a proper train dataset
    eval_dataset=dataset,   # Provide a proper eval dataset
)

trainer.train()
trainer

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at timpal0l/mdeberta-v3-base-squad2 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Development\Python311\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
# Saving Your Model
model.save_pretrained('./mistral_model')
tokenizer.save_pretrained('./mistral_model')

In [ ]:
# Using the Model for Inference
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity between two embeddings
similarity = cosine_similarity(cls_embeddings[0].unsqueeze(0), cls_embeddings[1].unsqueeze(0))
print("Cosine similarity:", similarity)